In [1]:
# Import packages

import pandas as pd 
import numpy as np 
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from joblib import dump, load

# test line, remove later

pd.set_option('display.max_columns', None)

In [2]:
nfl = pd.read_csv('../../data/nfl_game_by_game_raw_stats.csv')

In [3]:
# Pull out only the qb related stats

qb = nfl.copy()
qb = qb[['season', 'week', 'team', 'opponent', 'score', 'opponent_score', 'qb', 'total_pass_attempts', 'total_passing_yards',
        'completions', 'passing_epa', 'pass_tds', 'air_yards', 'yards_after_catch', 'air_epa', 'yac_epa', 'avg_cpoe',
        'qb_epa', 'qb_rush_yards', 'qb_rushing_epa', 'qb_rush_tds', 'sacks_taken_qb', 'qb_hits_taken_qb', 'fumbles_qb',
        'lost_fumbles_qb', 'interceptions_thrown_qb']]
qb

,season,week,team,opponent,score,opponent_score,qb,total_pass_attempts,total_passing_yards,completions,passing_epa,pass_tds,air_yards,yards_after_catch,air_epa,yac_epa,avg_cpoe,qb_epa,qb_rush_yards,qb_rushing_epa,qb_rush_tds,sacks_taken_qb,qb_hits_taken_qb,fumbles_qb,lost_fumbles_qb,interceptions_thrown_qb
0,2014,1,ARI,LAC,18,17,C. Palmer,38,304.0,24.0,6.098425,2.0,196.0,108.0,10.800692,13.884696,3.160065,7.779734,30.0,1.681309,0.0,2.0,4.0,1.0,1.0,0.0
1,2014,1,ATL,NO,37,34,M. Ryan,42,448.0,31.0,17.253965,3.0,253.0,195.0,8.815695,16.383532,6.128030,24.116820,15.0,0.272050,0.0,1.0,4.0,1.0,1.0,0.0
2,2014,1,BAL,CIN,16,23,J. Flacco,62,345.0,35.0,-4.871386,1.0,221.0,124.0,10.962930,14.313325,-11.889451,-3.078329,7.0,1.793058,0.0,3.0,6.0,0.0,0.0,1.0
3,2014,1,BUF,CHI,23,20,E. Manuel,22,173.0,16.0,0.368607,1.0,93.0,80.0,1.909633,9.530254,11.150598,0.591515,23.0,0.222908,1.0,1.0,3.0,0.0,0.0,1.0
4,2014,1,CAR,TB,20,14,D. Anderson,33,230.0,24.0,8.423189,2.0,160.0,70.0,6.298791,10.724494,9.822589,10.352312,10.0,1.929123,0.0,1.0,3.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5537,2022,10,TEN,DEN,17,10,R. Tannehill,36,255.0,19.0,4.792479,2.0,127.0,128.0,6.370534,16.488266,-15.691849,6.116046,12.0,1.323566,0.0,1.0,4.0,0.0,0.0,0.0
5538,2022,10,WAS,PHI,32,21,T. Heinicke,27,211.0,17.0,2.815080,0.0,141.0,70.0,8.408582,9.666575,6.855554,3.744092,14.0,0.929012,0.0,3.0,4.0,2.0,1.0,1.0
5539,2022,11,GB,TEN,17,27,A. Rodgers,40,229.0,24.0,2.710084,2.0,137.0,90.0,8.515738,8.720034,-0.817273,2.710084,0.0,0.000000,0.0,1.0,3.0,0.0,0.0,0.0
5540,2022,11,TEN,GB,27,17,D. Henry,2,4.0,2.0,0.466457,1.0,3.0,1.0,0.896837,-0.430380,46.528168,0.466457,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Want to remove any instances of trick plays, one or two play qb subs, etc.
# Filtering to only qbs that threw at least 10 passes in the game (may need to adjust this number at some point, 
# or find a better way to account for qb benchings/replacements/injuries)

qb = qb[qb.total_pass_attempts >= 10]

# Removing ties 

qb = qb[qb.score != qb.opponent_score]

# Add column to determine if qb won the game or not 

# qb['qb_win'] = qb.apply(lambda x: 1 if x.score > x.opponent_score else 0, axis=1)

# Going to do something similar to rb value system, where instead of predicting qb win you predict whether or not 
# the team scores more than 23 points

qb['qb_win'] = qb.apply(lambda x: 1 if x.score > 23 else 0, axis=1)

# Adding additional metrics

qb['passing_epa_per_attempt'] = qb.passing_epa/qb.total_pass_attempts
qb['yards_per_attempt'] = qb.total_passing_yards/qb.total_pass_attempts
qb['touchdowns_per_attempt'] = qb.pass_tds/qb.total_pass_attempts
qb['completion_percentage'] = qb.completions/qb.total_pass_attempts
qb['total_qb_plays'] = qb.total_pass_attempts + qb.total_qb_rush_attempts
qb['total_epa_per_play'] = qb.qb_epa/qb.total_qb_plays

qb

,season,week,team,opponent,score,opponent_score,qb,total_pass_attempts,total_passing_yards,completions,passing_epa,pass_tds,air_yards,yards_after_catch,air_epa,yac_epa,avg_cpoe,qb_epa,qb_rush_yards,qb_rushing_epa,qb_rush_tds,sacks_taken_qb,qb_hits_taken_qb,fumbles_qb,lost_fumbles_qb,interceptions_thrown_qb,qb_win,passing_epa_per_attempt,yards_per_attempt,touchdowns_per_attempt,completion_percentage
0,2014,1,ARI,LAC,18,17,C. Palmer,38,304.0,24.0,6.098425,2.0,196.0,108.0,10.800692,13.884696,3.160065,7.779734,30.0,1.681309,0.0,2.0,4.0,1.0,1.0,0.0,0,0.160485,8.000000,0.052632,0.631579
1,2014,1,ATL,NO,37,34,M. Ryan,42,448.0,31.0,17.253965,3.0,253.0,195.0,8.815695,16.383532,6.128030,24.116820,15.0,0.272050,0.0,1.0,4.0,1.0,1.0,0.0,1,0.410809,10.666667,0.071429,0.738095
2,2014,1,BAL,CIN,16,23,J. Flacco,62,345.0,35.0,-4.871386,1.0,221.0,124.0,10.962930,14.313325,-11.889451,-3.078329,7.0,1.793058,0.0,3.0,6.0,0.0,0.0,1.0,0,-0.078571,5.564516,0.016129,0.564516
3,2014,1,BUF,CHI,23,20,E. Manuel,22,173.0,16.0,0.368607,1.0,93.0,80.0,1.909633,9.530254,11.150598,0.591515,23.0,0.222908,1.0,1.0,3.0,0.0,0.0,1.0,0,0.016755,7.863636,0.045455,0.727273
4,2014,1,CAR,TB,20,14,D. Anderson,33,230.0,24.0,8.423189,2.0,160.0,70.0,6.298791,10.724494,9.822589,10.352312,10.0,1.929123,0.0,1.0,3.0,1.0,0.0,0.0,0,0.255248,6.969697,0.060606,0.727273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5536,2022,10,TB,SEA,21,16,T. Brady,29,258.0,22.0,15.308453,2.0,156.0,102.0,8.918437,15.533023,10.649817,15.308453,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,1.0,0,0.527878,8.896552,0.068966,0.758621
5537,2022,10,TEN,DEN,17,10,R. Tannehill,36,255.0,19.0,4.792479,2.0,127.0,128.0,6.370534,16.488266,-15.691849,6.116046,12.0,1.323566,0.0,1.0,4.0,0.0,0.0,0.0,0,0.133124,7.083333,0.055556,0.527778
5538,2022,10,WAS,PHI,32,21,T. Heinicke,27,211.0,17.0,2.815080,0.0,141.0,70.0,8.408582,9.666575,6.855554,3.744092,14.0,0.929012,0.0,3.0,4.0,2.0,1.0,1.0,1,0.104262,7.814815,0.000000,0.629630
5539,2022,11,GB,TEN,17,27,A. Rodgers,40,229.0,24.0,2.710084,2.0,137.0,90.0,8.515738,8.720034,-0.817273,2.710084,0.0,0.000000,0.0,1.0,3.0,0.0,0.0,0.0,0,0.067752,5.725000,0.050000,0.600000


In [5]:
# Function to get relevant columns for modeling

# Note 1: Not using air yards or yac yards at the moment - since it's just predicting a win it won't be any different than 
# just the total passing yards idt. Might want a future adjustment to maybe raise/lower qb value based on air yards vs yac 
# yards pct

# Note 2: Going to use passing epa and qb rush epa seperately, rather than total qb epa

# Note 3: Using qb sacks and qb hits as predictive features. Not totally on the qb, but maybe theres some logic to 
# some qbs hold the ball too long and take too many sacks
# UPDATE: No longer using qb sacks and qb hits since they are pretty big factors in the qb value model, and since they are
# not entirely on the qb i think its not fair to add that, maybe worth adding in a separate linemen grading category

from qb_value_helper_functions import get_features, get_label

In [6]:
# Test Sets of last 8 weeks of 2021 season and whatever we have from 2022 season

# test_2021 = qb[(qb.season == 2021) & (qb.week >= 9)]
# test_2021_x = get_features(test_2021)
# test_2021_y = get_label(test_2021)

# test_2022 = qb[qb.season == 2022]
# test_2022_x = get_features(test_2022)
# test_2022_y = get_label(test_2022)

test = qb[((qb.season == 2021) & (qb.week >= 9)) | (qb.season == 2022)]
test_x = get_features(test)
test_y = get_label(test)

train = qb[(qb.season < 2021) | ((qb.season == 2021) & (qb.week < 9))]
train_features = get_features(train)
train_labels = get_label(train)

qb.to_csv('qb_value_data/qb.csv')
train.to_csv('qb_value_data/train.csv')
test.to_csv('qb_value_data/test.csv')

train_x, val_x, train_y, val_y = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)

In [7]:
def train_model(model, train_x, train_y, val_x, val_y):
    clf = model.fit(train_x, train_y.to_numpy().flatten())
    
    train_preds = clf.predict(train_x)
    val_preds = clf.predict(val_x)
    
    train_acc = accuracy_score(train_y, train_preds)
    val_acc = accuracy_score(val_y, val_preds)
    
    print('Training Accuracy: {}'.format(train_acc))
    print('Validation Accuracy: {}'.format(val_acc))
    
    return clf

def test_model(clf, test_x, test_y):
    preds = clf.predict(test_x)
    probs = clf.predict_proba(test_x)[:,1]
    
    acc = accuracy_score(test_y, preds)
    
    print('Testing Accuracy: {}'.format(acc))
    
    return preds, probs

In [8]:
# Logistic Regression Model

model = LogisticRegression(max_iter=1000, random_state=71)
lr_model = train_model(model, train_x, train_y, val_x, val_y)

test_preds, test_probs = test_model(lr_model, test_x, test_y)

dump(lr_model, 'qb_value_saved_models/logistic_regression.joblib')

Training Accuracy: 0.7664634146341464
Validation Accuracy: 0.7624847746650426
Testing Accuracy: 0.7453416149068323


['qb_value_saved_models/logistic_regression.joblib']